In [30]:
import json
import os
import csv
from datetime import datetime, timezone
from zoneinfo import ZoneInfo  # Use pytz if on Python <3.9

# Set your parameters
input_file = r"D:\reddit\subreddits23\wallstreetbets_comments\wallstreetbets_comments"
output_file = r"D:\reddit\output\filtered_comments.csv"
from_date = datetime(2021, 1, 1, tzinfo=ZoneInfo("America/New_York"))
to_date = datetime(2023, 12, 31, 23, 59, 59, tzinfo=ZoneInfo("America/New_York"))

# Ensure the output directory exists
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def process_comments(input_file, output_file, from_date, to_date):
    with open(input_file, 'r', encoding='utf-8') as file_in, open(output_file, 'w', encoding='utf-8', newline='') as file_out:
        writer = csv.writer(file_out, quoting=csv.QUOTE_MINIMAL)
        
        # Writing CSV header for the comments dataset
        writer.writerow(["id", "score", "date", "time", "author", "parent_id", "link_id", "body"])

        for line in file_in:
            try:
                obj = json.loads(line.strip())  # Load each line as a JSON object
                
                # Convert created_utc to a timezone-aware datetime in UTC
                created_utc = datetime.fromtimestamp(int(obj['created_utc']), tz=timezone.utc)
                
                # Convert UTC datetime to Eastern Time (ET)
                created_et = created_utc.astimezone(ZoneInfo("America/New_York"))

                # Filter by date range in ET
                if from_date <= created_et <= to_date:
                    # Extract fields for the comments dataset
                    comment_id = obj.get('id', '')
                    score = obj.get('score', '')
                    date = created_et.strftime("%Y-%m-%d")
                    time = created_et.strftime("%H:%M:%S")  # Exact time in seconds
                    author = obj.get('author', '')
                    parent_id = obj.get('parent_id', '')
                    # Remove the 't3_' prefix from link_id for later matching
                    link_id = obj.get('link_id', '').replace('t3_', '')
                    body = obj.get('body', '')

                    # Write to CSV
                    writer.writerow([comment_id, score, date, time, author, parent_id, link_id, body])

            except json.JSONDecodeError as e:
                print(f"Skipping line due to JSON decode error: {e}")

# Run the processing function
process_comments(input_file, output_file, from_date, to_date)

In [31]:
import pandas as pd

# Define the file paths and create separate CSV writers for each year
input_file = 'D:/reddit/output/filtered_comments.csv'
output_files = {
    2021: 'D:/reddit/output/filtered_comments_2021.csv',
    2022: 'D:/reddit/output/filtered_comments_2022.csv',
    2023: 'D:/reddit/output/filtered_comments_2023.csv'
}

# Initialize files for each year and write headers with full columns
for year, file_path in output_files.items():
    with open(file_path, 'w', encoding='utf-8', newline='') as file:
        file.write("id,score,date,time,author,parent_id,link_id,body\n")  # Writing header

# Process the input file in chunks
chunk_size = 100000  # Adjust the chunk size based on available memory

for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=["date"]):
    # Drop rows with invalid dates (if any)
    chunk = chunk.dropna(subset=['date'])

    # Ensure columns are in the correct order to avoid misalignment
    chunk = chunk[['id', 'score', 'date', 'time', 'author', 'parent_id', 'link_id', 'body']]

    # Extract the year from the 'date' column
    chunk['year'] = chunk['date'].dt.year

    # Filter each chunk by year and append to the respective output file
    for year in [2021, 2022, 2023]:
        df_year = chunk[chunk['year'] == year]
        if not df_year.empty:
            with open(output_files[year], 'a', encoding='utf-8', newline='') as file:
                df_year.to_csv(file, columns=['id', 'score', 'date', 'time', 'author', 'parent_id', 'link_id', 'body'], header=False, index=False)

print("Data successfully split by year into separate files.")

Data successfully split by year into separate files.


In [32]:
import pandas as pd

# Path to the 2023 comments data
file_path_2023 = 'D:/reddit/output/filtered_comments_2023.csv'

# Load a sample of the 2023 data to verify it
df_2023_sample = pd.read_csv(file_path_2023, nrows=100)

In [33]:
df_2023_sample

,id,score,date,time,author,parent_id,link_id,body
0,j2gxjta,4,2023-01-01,00:00:02,ninkorn,t3_zzbd4p,zzbd4p,# HAPPY NEW YEAR FELLOW REGARDS\n\nDON'T CHANG...
1,j2gxjv4,8,2023-01-01,00:00:02,TheLastOpus,t3_100bodo,100bodo,I would do the onlyfans route if i could.....
2,j2gxjzq,1,2023-01-01,00:00:05,ABena2t,t1_j2gwm4l,zzlf9r,lol
3,j2gxk3d,2,2023-01-01,00:00:06,mathboom123,t1_j2gxfo0,zzbd4p,Me
4,j2gxk7i,1,2023-01-01,00:00:08,NarcolepticTreesnake,t3_100bujd,100bujd,Run from here as fast as you can and light $20...
...,...,...,...,...,...,...,...,...
95,j2gy9ct,6,2023-01-01,00:07:05,ipfreelytv,t3_zzbd4p,zzbd4p,no losses this year hell ya
96,j2gy9qn,1,2023-01-01,00:07:11,VisualMod,t3_100ce61,100ce61,I am a bot from /r/wallstreetbets. Your submis...
97,j2gya2n,0,2023-01-01,00:07:17,Liquid_Skeet,t3_zzbd4p,zzbd4p,Victoria Justice should make a sex tape with h...
98,j2gyalo,1,2023-01-01,00:07:26,_Kenway,t3_zzbd4p,zzbd4p,"working on a meme 2022 video recap, kinda late..."


In [ ]:
del df_2023_sample

In [36]:
import json
import os
import csv
from datetime import datetime, timezone
from zoneinfo import ZoneInfo  # Use pytz if on Python <3.9

# Set your parameters
input_file = r"D:\reddit\subreddits23\wallstreetbets_submissions\wallstreetbets_submissions"
output_file = r"D:\reddit\output\filtered_submissions.csv"
from_date = datetime(2021, 1, 1, tzinfo=ZoneInfo("America/New_York"))
to_date = datetime(2023, 12, 31, 23, 59, 59, tzinfo=ZoneInfo("America/New_York"))

# Ensure the output directory exists
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def process_file(input_file, output_file, from_date, to_date):
    with open(input_file, 'r', encoding='utf-8') as file_in, open(output_file, 'w', encoding='utf-8', newline='') as file_out:
        writer = csv.writer(file_out, quoting=csv.QUOTE_MINIMAL)
        
        # Writing CSV header based on required fields
        writer.writerow(["id", "score", "date", "time", "title", "author", "permalink", "selftext", "url"])

        for line in file_in:
            try:
                obj = json.loads(line.strip())  # Load each line as a JSON object
                
                # Convert created_utc to a timezone-aware datetime in UTC
                created_utc = datetime.fromtimestamp(int(obj['created_utc']), tz=timezone.utc)
                
                # Convert UTC datetime to Eastern Time (ET)
                created_et = created_utc.astimezone(ZoneInfo("America/New_York"))

                # Filter by date range in ET
                if from_date <= created_et <= to_date:
                    # Extract fields based on requirements
                    submission_id = obj.get('id', '')
                    score = obj.get('score', '')  # Assuming 'score' is available; adjust if needed
                    date = created_et.strftime("%Y-%m-%d")
                    time = created_et.strftime("%H:%M:%S")  # Exact time in seconds
                    title = obj.get('title', '')
                    author = obj.get('author', '')
                    permalink = obj.get('permalink', '')
                    selftext = obj.get('selftext', '')
                    url = obj.get('url', '')

                    # Write to CSV
                    writer.writerow([submission_id, score, date, time, title, author, permalink, selftext, url])

            except json.JSONDecodeError as e:
                print(f"Skipping line due to JSON decode error: {e}")

# Run the processing function
process_file(input_file, output_file, from_date, to_date)

In [37]:
import pandas as pd

# Define the file paths and create separate CSV writers for each year
input_file = 'D:/reddit/output/filtered_submissions.csv'
output_files = {
    2021: 'D:/reddit/output/filtered_submissions_2021.csv',
    2022: 'D:/reddit/output/filtered_submissions_2022.csv',
    2023: 'D:/reddit/output/filtered_submissions_2023.csv'
}

# Initialize files for each year and write headers with full columns
for year, file_path in output_files.items():
    with open(file_path, 'w', encoding='utf-8', newline='') as file:
        file.write("id,score,date,time,title,author,permalink,selftext,url\n")  # Writing header

# Process the input file in chunks
chunk_size = 100000  # Adjust the chunk size based on available memory

for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=["date"]):
    # Drop rows with invalid dates (if any)
    chunk = chunk.dropna(subset=['date'])

    # Ensure columns are in the correct order to avoid misalignment
    chunk = chunk[['id', 'score', 'date', 'time', 'title', 'author', 'permalink', 'selftext', 'url']]

    # Extract the year from the 'date' column
    chunk['year'] = chunk['date'].dt.year

    # Filter each chunk by year and append to the respective output file
    for year in [2021, 2022, 2023]:
        df_year = chunk[chunk['year'] == year]
        if not df_year.empty:
            with open(output_files[year], 'a', encoding='utf-8', newline='') as file:
                df_year.to_csv(file, columns=['id', 'score', 'date', 'time', 'title', 'author', 'permalink', 'selftext', 'url'], header=False, index=False)

print("Submissions data successfully split by year into separate files.")

Submissions data successfully split by year into separate files.


In [38]:
import pandas as pd

# Path to the 2023 comments data
file_path_2023 = 'D:/reddit/output/filtered_submissions_2023.csv'

# Load a sample of the 2023 data to verify it
df_2023_sample = pd.read_csv(file_path_2023, nrows=100)

In [39]:
df_2023_sample

,id,score,date,time,title,author,permalink,selftext,url
0,100cbuj,6,2023-01-01,00:02:48,Lazymeme.jpg,[deleted],/r/wallstreetbets/comments/100cbuj/lazymemejpg/,[removed],NaN
1,100ce61,1,2023-01-01,00:07:08,Brainchip Stock To $2??,Firm-Blueberry-5919,/r/wallstreetbets/comments/100ce61/brainchip_s...,[removed],https://www.reddit.com/r/wallstreetbets/commen...
2,100ceov,12,2023-01-01,00:08:03,"2023 here we come, hope it turns out better th...",[deleted],/r/wallstreetbets/comments/100ceov/2023_here_w...,[removed],NaN
3,100cf5c,1,2023-01-01,00:08:56,Looks like OSU discovered options...,[deleted],/r/wallstreetbets/comments/100cf5c/looks_like_...,[removed],NaN
4,100ch63,108,2023-01-01,00:12:28,Doesn't feel right if I don't post this here e...,SynapseCero,/r/wallstreetbets/comments/100ch63/doesnt_feel...,"Well, end of the year, how could I not post th...",https://www.reddit.com/r/wallstreetbets/commen...
...,...,...,...,...,...,...,...,...,...
95,100musf,1,2023-01-01,11:14:52,More AI Art: Jerome Powell Eating Barbecue Bul...,[deleted],/r/wallstreetbets/comments/100musf/more_ai_art...,[removed],NaN
96,100mut5,0,2023-01-01,11:14:54,Where to paper trade options?,saippuakauppias,/r/wallstreetbets/comments/100mut5/where_to_pa...,[removed],https://www.reddit.com/r/wallstreetbets/commen...
97,100n8rj,1,2023-01-01,11:33:36,I’ll make it back this year…,[deleted],/r/wallstreetbets/comments/100n8rj/ill_make_it...,[removed],NaN
98,100na2e,282,2023-01-01,11:35:19,Hoping these calls will bring me back,[deleted],/r/wallstreetbets/comments/100na2e/hoping_thes...,[deleted],NaN


In [49]:
import pandas as pd
import csv

# Define file paths for submissions and comments by year
submissions_files = {
    2021: 'D:/reddit/output/filtered_submissions_2021.csv',
    2022: 'D:/reddit/output/filtered_submissions_2022.csv',
    2023: 'D:/reddit/output/filtered_submissions_2023.csv'
}
comments_files = {
    2021: 'D:/reddit/output/filtered_comments_2021.csv',
    2022: 'D:/reddit/output/filtered_comments_2022.csv',
    2023: 'D:/reddit/output/filtered_comments_2023.csv'
}

# Output files for merged data and unmatched comments
output_files = {
    2021: 'D:/reddit/output/flattened_submissions_with_comments_2021.csv',
    2022: 'D:/reddit/output/flattened_submissions_with_comments_2022.csv',
    2023: 'D:/reddit/output/flattened_submissions_with_comments_2023.csv'
}
unmatched_files = {
    2021: 'D:/reddit/output/unmatched_comments_2021.csv',
    2022: 'D:/reddit/output/unmatched_comments_2022.csv',
    2023: 'D:/reddit/output/unmatched_comments_2023.csv'
}
error_log_file = 'D:/reddit/output/error_log.csv'  # Error log file to keep track of problematic rows

# Initialize a list to keep track of errors and dropped rows
error_log = []
drop_log = []

# Process each year separately
for year in [2021, 2022, 2023]:
    # Load submissions with error handling
    try:
        submissions_df = pd.read_csv(submissions_files[year], on_bad_lines='skip')
    except pd.errors.ParserError as e:
        error_log.append({'file': submissions_files[year], 'year': year, 'error': str(e)})
        print(f"Error reading submissions file for {year}: {e}")
        continue

    # Rename columns in submissions to differentiate them when merged with comments
    submissions_df = submissions_df.rename(columns={
        "id": "submission_id",
        "score": "submission_score",
        "date": "submission_date",
        "time": "submission_time",
        "title": "submission_title",
        "author": "submission_author",
        "permalink": "submission_permalink",
        "selftext": "submission_selftext",
        "url": "submission_url"
    })

    # Load comments with error handling
    try:
        comments_df = pd.read_csv(comments_files[year], on_bad_lines='skip')
    except pd.errors.ParserError as e:
        error_log.append({'file': comments_files[year], 'year': year, 'error': str(e)})
        print(f"Error reading comments file for {year}: {e}")
        continue

    # Count the original number of rows in comments
    original_row_count = len(comments_df)

    # Remove rows with NaN values in `parent_id`
    comments_df = comments_df.dropna(subset=['parent_id'])

    # Count and log the number of dropped rows
    dropped_row_count = original_row_count - len(comments_df)
    drop_log.append({'year': year, 'dropped_rows': dropped_row_count})
    print(f"Dropped {dropped_row_count} rows with NaN parent_id for year {year}")

    # Filter comments to include only those with `parent_id` pointing to a submission (i.e., starts with "t3_")
    comments_df = comments_df[comments_df['parent_id'].str.startswith('t3_')].copy()

    # Clean up `parent_id` by removing the "t3_" prefix
    comments_df['parent_id'] = comments_df['parent_id'].str.replace('t3_', '')

    # Rename columns in comments for clarity
    comments_df = comments_df.rename(columns={
        "id": "comment_id",
        "score": "comment_score",
        "date": "comment_date",
        "time": "comment_time",
        "author": "comment_author",
        "body": "comment_body"
    })

    # Merge comments with their associated submission data (left join to keep all comments)
    merged_df = comments_df.merge(submissions_df, left_on='parent_id', right_on='submission_id', how='left')

    # Separate unmatched comments (those with NaN in `submission_id` column)
    unmatched_comments_df = merged_df[merged_df['submission_id'].isna()]

    # Log unmatched comments to a separate CSV file
    if not unmatched_comments_df.empty:
        unmatched_comments_df.to_csv(unmatched_files[year], index=False)
        print(f"Unmatched comments for {year} saved to {unmatched_files[year]}")
    else:
        print(f"No unmatched comments for {year}.")

    # Filter out unmatched comments from the merged data (optional)
    matched_comments_df = merged_df.dropna(subset=['submission_id'])

    # Save the matched data to a flat CSV file
    matched_comments_df.to_csv(output_files[year], index=False)
    print(f"Flattened data for {year} saved to {output_files[year]}")

# Save the error log to a CSV file if there were any errors
if error_log:
    pd.DataFrame(error_log).to_csv(error_log_file, index=False)
    print(f"Error log saved to {error_log_file}")
else:
    print("No errors encountered during processing.")

# Save the drop log to a CSV file to record the dropped rows
drop_log_file = 'D:/reddit/output/drop_log.csv'
pd.DataFrame(drop_log).to_csv(drop_log_file, index=False)
print(f"Drop log saved to {drop_log_file}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_67176\438859393.py:58: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_df = pd.read_csv(comments_files[year], on_bad_lines='skip')


Dropped 9 rows with NaN parent_id for year 2021
Unmatched comments for 2021 saved to D:/reddit/output/unmatched_comments_2021.csv
Flattened data for 2021 saved to D:/reddit/output/flattened_submissions_with_comments_2021.csv
Dropped 0 rows with NaN parent_id for year 2022
Unmatched comments for 2022 saved to D:/reddit/output/unmatched_comments_2022.csv
Flattened data for 2022 saved to D:/reddit/output/flattened_submissions_with_comments_2022.csv
Dropped 0 rows with NaN parent_id for year 2023
Unmatched comments for 2023 saved to D:/reddit/output/unmatched_comments_2023.csv
Flattened data for 2023 saved to D:/reddit/output/flattened_submissions_with_comments_2023.csv
No errors encountered during processing.
Drop log saved to D:/reddit/output/drop_log.csv
